In [ ]:
import pandas as pd

# Leer la hoja "Element Forces - Columns" con encabezados en la segunda fila
df_excel = pd.read_excel("Libro1.xlsx", sheet_name="Element Forces - Columns", engine="openpyxl", header=1)

# Filtrar columnas relevantes y eliminar filas vacías
df_puntos = df_excel[['Output Case', 'P', 'M2', 'M3']].dropna()

# Multiplicar los valores por -1
df_puntos['P'] = -df_puntos['P']
df_puntos['M2'] = -df_puntos['M2']
df_puntos['M3'] = -df_puntos['M3']

# Guardar los datos en un archivo CSV para uso posterior
df_puntos.to_csv("puntos_interaccion.csv", index=False)

print("✅ Datoss cargados y transformados correctamente. Guardados en 'puntos_interaccion.csv'.")



✅ Datos cargados y transformados correctamente. Guardados en 'puntos_interaccion.csv'.


In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import sqrt, pi
from ipywidgets import interact, FloatSlider, IntSlider
from IPython.display import display, HTML

# Mostrar todas las columnas sin truncar
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

# === Cargar datos desde el archivo Excel ===
df_excel = pd.read_excel("Libro1.xlsx", sheet_name="Element Forces - Columns", engine="openpyxl", header=1)
df_puntos = df_excel[['Output Case', 'P', 'M2', 'M3']].dropna()
df_puntos['P'] = -df_puntos['P']
df_puntos['M2'] = -df_puntos['M2']
df_puntos['M3'] = -df_puntos['M3']
P_excel = df_puntos['P'].values
M2_excel = df_puntos['M2'].values
M3_excel = df_puntos['M3'].values
cases_excel = df_puntos['Output Case'].values

# === Funciones auxiliares ===
def propiedades_materiales(f_c):
    Ec = 4700 * sqrt(f_c)
    β1 = np.clip(0.85 - 0.05 * (f_c - 28) / 7, 0.65, 0.85)
    return Ec, β1

def refuerzo_por_direccion(n, extremo_valor, As_bar):
    ref = np.full((1, n), 2)
    ref[0, 0] = ref[0, -1] = extremo_valor
    return ref * As_bar

def posiciones(n, lado, rec):
    lado_util = lado - 2 * rec
    delta = lado_util / (n - 1)
    return np.array([[rec + i * delta for i in range(n)]])

def eje_neutro(lado):
    return np.linspace(0.01 * lado, 1.0 * lado, 100).reshape(-1, 1)

def analizar_direccion(c, d, As, lado, lado_ort, f_c, fy, Ey, β1, εcu, εy, εyu, phi_Pn_max, phi_Tn_max, Pn_max, Tn_max):
    ε = εcu * (c - d) / c
    σ = np.clip(Ey * ε, -fy, fy)
    Fs = σ * As
    Fc = 0.85 * f_c * β1 * c * lado_ort
    Pn = np.sum(Fs, axis=1).reshape(-1,1) + Fc

    brazo_s = c - d
    brazo_c = c - β1 * c / 2
    brazo_n = lado / 2 - c

    Ms = np.sum(Fs * brazo_s, axis=1).reshape(-1,1)
    Mc = Fc * brazo_c
    Mn = Ms + Mc + Pn * brazo_n

    εu = ε[:, -1].reshape(-1,1)
    φ = np.clip(0.65 + (abs(εu) - abs(εy)) * (0.90 - 0.65) / (abs(εyu) - abs(εy)), 0.65, 0.90)
    φPn = φ * Pn
    φPn = np.clip(φPn, None, phi_Pn_max)
    φMn = φ * Mn

    Pn = np.vstack([[Tn_max], Pn, [Pn_max]])
    φPn = np.vstack([[phi_Tn_max], φPn, [phi_Pn_max]])
    Mn = np.vstack([[0], Mn, [0]])
    φMn = np.vstack([[0], φMn, [0]])

    filas = []
    n_barras = d.shape[1]
    for i in range(len(c)):
        fila = {'c': c[i, 0]}
        for j in range(n_barras):
            fila[f'εs{j+1}'] = ε[i, j]
        for j in range(n_barras):
            fila[f'fs{j+1}'] = σ[i, j]
        for j in range(n_barras):
            fila[f'F{j+1}'] = Fs[i, j] * 1000
        fila['Fc'] = Fc[i, 0] * 1000
        for j in range(n_barras):
            fila[f'b{j+1}'] = brazo_s[i, j]
        fila['bc'] = brazo_c[i, 0]
        fila['bP'] = brazo_n[i, 0]
        fila['Pn'] = Pn[i, 0] * 1000
        fila['Mn'] = Mn[i, 0] * 1000
        fila['φ'] = φ[i, 0]
        fila['φMn'] = φMn[i, 0] * 1000
        fila['φPn'] = φPn[i, 0] * 1000
        filas.append(fila)
    df = pd.DataFrame(filas)
    return Mn, Pn, φMn, φPn, df

def plot_interaction(Mn, Pn, phi_Mn, phi_Pn, dir_label, color1, color2, puntos_M, puntos_P, etiquetas):
    plt.figure(figsize=(9,6))
    plt.plot(Mn, Pn, color=color1, label=f'P - M Nominal')
    plt.plot(phi_Mn, phi_Pn, color=color2, linestyle='--', label=f'P - M Reducido (φ)')
    plt.scatter(puntos_M, puntos_P, color='black', marker='o', label='Solicitaciones')
    plt.xlabel(f'M{dir_label} [kN·m]')
    plt.ylabel('P [kN]')
    plt.title(f'Diagrama de Interacción - Dirección {dir_label}')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

def mostrar_tabla_scroll(df):
    html_table = df.to_html()
    scrollable_html = f"""
    <div style='overflow-x:auto; overflow-y:auto; max-height:400px; max-width:100%; border:1px solid #ccc;'>
      {html_table}
    </div>
    """
    display(HTML(scrollable_html))

# === Widget interactivo ===
@interact(
    f_c=FloatSlider(value=28, min=20, max=50, step=1, description="f'c (MPa)"),
    fy=FloatSlider(value=420, min=280, max=600, step=10, description='fy (MPa)'),
    rec=FloatSlider(value=0.06, min=0.02, max=0.1, step=0.005, description='Recubrimiento (m)'),
    lx=FloatSlider(value=0.5, min=0.3, max=1.0, step=0.05, description='Lado X (m)'),
    ly=FloatSlider(value=0.95, min=0.3, max=1.5, step=0.05, description='Lado Y (m)'),
    bar_number=IntSlider(value=10, min=3, max=12, step=1, description='Barra No.'),
    nx=IntSlider(value=5, min=2, max=10, step=1, description='Barras X'),
    ny=IntSlider(value=5, min=2, max=10, step=1, description='Barras Y')
)
def actualizar(f_c, fy, rec, lx, ly, bar_number, nx, ny):
    Ec, β1 = propiedades_materiales(f_c)
    Ey = 200_000
    εcu, εy, εyu = 0.003, 0.0021, 0.005

    Ag = lx * ly
    bar_diameter = bar_number / 8 * 0.0254
    As_bar = pi * (bar_diameter / 2)**2
    n_barras = 2 * (nx + ny - 2)
    As_total = n_barras * As_bar

    Pn_max = 0.85 * f_c * Ag + fy * As_total
    Tn_max = -As_total * fy
    phi_Pn_max = 0.75 * 0.65 * Pn_max
    phi_Tn_max = 0.90 * Tn_max

    As_x = refuerzo_por_direccion(nx, ny, As_bar)
    As_y = refuerzo_por_direccion(ny, nx, As_bar)
    d_x = posiciones(nx, lx, rec)
    d_y = posiciones(ny, ly, rec)
    cx = eje_neutro(lx)
    cy = eje_neutro(ly)

    Mn_x, Pn_x, phi_Mn_x, phi_Pn_x, df_x = analizar_direccion(cx, d_x, As_x, lx, ly, f_c, fy, Ey, β1, εcu, εy, εyu, phi_Pn_max, phi_Tn_max, Pn_max, Tn_max)
    Mn_y, Pn_y, phi_Mn_y, phi_Pn_y, df_y = analizar_direccion(cy, d_y, As_y, ly, lx, f_c, fy, Ey, β1, εcu, εy, εyu, phi_Pn_max, phi_Tn_max, Pn_max, Tn_max)

    Mn_x_r = np.concatenate((-Mn_x[::-1], Mn_x)) * 1000
    Pn_x_r = np.concatenate((Pn_x[::-1], Pn_x)) * 1000
    phi_Mn_x_r = np.concatenate((-phi_Mn_x[::-1], phi_Mn_x)) * 1000
    phi_Pn_x_r = np.concatenate((phi_Pn_x[::-1], phi_Pn_x)) * 1000

    Mn_y_r = np.concatenate((-Mn_y[::-1], Mn_y)) * 1000
    Pn_y_r = np.concatenate((Pn_y[::-1], Pn_y)) * 1000
    phi_Mn_y_r = np.concatenate((-phi_Mn_y[::-1], phi_Mn_y)) * 1000
    phi_Pn_y_r = np.concatenate((phi_Pn_y[::-1], phi_Pn_y)) * 1000

    plot_interaction(Mn_x_r, Pn_x_r, phi_Mn_x_r, phi_Pn_x_r, '2 (X)', 'blue', 'red', M2_excel, P_excel, cases_excel)
    plot_interaction(Mn_y_r, Pn_y_r, phi_Mn_y_r, phi_Pn_y_r, '3 (Y)', 'green', 'orange', M3_excel, P_excel, cases_excel)

    print("\nTabla de resultados - Dirección X")
    mostrar_tabla_scroll(df_x.round(4))

    print("\nTabla de resultados - Dirección Y")
    mostrar_tabla_scroll(df_y.round(4))


interactive(children=(FloatSlider(value=28.0, description="f'c (MPa)", max=50.0, min=20.0, step=1.0), FloatSli…